In [4]:
### Use with quandl environment

import twint
#import asyncio
import nest_asyncio
from datetime import datetime
import pytz
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 0)

nest_asyncio.apply()
#asyncio.set_event_loop(asyncio.new_event_loop())

todayUTC=datetime.today()

# dd/mm/YY H:M:S
to_zone = pytz.timezone('Asia/Bangkok')
today=todayUTC.astimezone(to_zone)

todayStr=today.strftime("%Y-%m-%d")+' 00:00:00'

c=twint.Config()

c.Hide_output = True
c.Limit=200
c.Since=todayStr
#c.Since='2020-05-09 00:00:00'
c.Pandas=True
#c.Near="ลาดพร้าว"
c.Search=r"#รถติด OR #อุบัติเหตุ"


twint.run.Search(c)

def available_columns():
    return twint.output.panda.Tweets_df.columns


def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

#available_columns()

df_pd = twint_to_pandas(["date","timezone","place","username", "tweet", "hashtags"])

#df_pd

In [5]:
from googlemaps import Client as GoogleMaps

myAPI=''

gmaps = GoogleMaps(myAPI)  # my account API, replace with yours

def LatLon_1(dfIn, colName):
    dfIn['lat'] = ""
    dfIn['lon'] = ""

    for x in range(len(dfIn)):
        dummy=dfIn[colName][x]
        #print(dummy)
        geocode_result = gmaps.geocode(dummy)
        #print(geocode_result)
        try:
            dfIn['lat'][x] = geocode_result[0]['geometry']['location']['lat']
            dfIn['lon'][x] = geocode_result[0]['geometry']['location']['lng']
        except:
            dfIn['lat'][x] = ''
            dfIn['lon'][x] = ''
    return dfIn


dfAcc=LatLon_1(df_pd,'tweet')

dateList=[]
weightL=[]
colorList=[]
for n in dfAcc['date']:
    stringDate = datetime.strptime(n, '%Y-%m-%d %H:%M:%S')
    tweetTime=stringDate.astimezone(to_zone)
    dateList.append(tweetTime)
    weightL.append(int(n[10:13]))
    #print(type(n), ' -- ',type(stringDate))

maxW=max(weightL)
weightList=[n/maxW for n in weightL]    
for n in weightList:
    if(n<=1 and n>=0.9):
        colorList.append('red')
    elif(n>=0.5 and n<0.9):
        colorList.append('yellow')
    else:
        colorList.append('green')

stringDf=pd.DataFrame(dateList, columns=['datetime'])
weightDf=pd.DataFrame(weightList, columns=['weight'])
colorDf=pd.DataFrame(colorList, columns=['color'])
dfAcc=pd.concat([dfAcc,stringDf,weightDf,colorDf],axis=1)


#dfAcc

In [10]:
from gmplot import gmplot
from matplotlib import pyplot as plt
import gmaps
import numpy as np
from ipywidgets.embed import embed_minimal_html

gmaps.configure(api_key=myAPI)
gmap = gmplot.GoogleMapPlotter(13.7464,100.646,6)
gmap.apikey = myAPI

dfAcc['lat'].replace('', np.nan, inplace=True)
dfAcc['lon'].replace('', np.nan, inplace=True)
dfAcc_1=dfAcc.dropna().copy().reset_index()

locations = dfAcc_1[['lat', 'lon']].copy()
weights = dfAcc_1['weight'].copy()
colors=dfAcc_1['color'].values.tolist()

figure_layout = {
    'width': '1200px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

bangkok_coordinates = (13.7464, 100.646)
fig = gmaps.figure(center=bangkok_coordinates,zoom_level=11, layout=figure_layout)
#fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

heatLayer=gmaps.heatmap_layer(locations, weights=weights)
heatLayer.opacity = 1.0 # make the heatmap transparent
#fig.add_layer(heatLayer)

latList=dfAcc_1['lat'].values.tolist()
lonList=dfAcc_1['lon'].values.tolist()
llData=list(zip(latList,lonList))
labelList=[]
for n in range(0,len(dfAcc_1)):
    labelList.append({"name":dfAcc_1['username'][n], "datetime":dfAcc_1['datetime'][n],
    "location":(dfAcc_1['lat'][n],dfAcc_1['lon'][n]), "tweet":dfAcc_1['tweet'][n]})
    
info_box_template = """
<dl>
<dt><b><p style="color:red;">username<p></b></dt><dd><p style="color:red;">{name}<p></dd>
<dt><b><p style="color:red;">datetime<p></b></dt><dd><p style="color:red;">{datetime}<p></dd>
<dt><b><p style="color:red;">tweet<p></b></dt><dd><p style="color:red;">{tweet}<p></dd>
</dl>
"""

plant_info = [info_box_template.format(**plant) for plant in labelList]

info_box_template_2 = """
username : {name}
datetime : {datetime}
tweet    :{tweet}
"""
plant_info_2 = [info_box_template_2.format(**plant) for plant in labelList]

markerLayer=gmaps.marker_layer(llData, info_box_content=plant_info, hover_text=plant_info_2)
fig.add_layer(markerLayer)


incLayer = gmaps.symbol_layer(
    locations, fill_color=colors, stroke_color=colors, scale=5
)
fig.add_layer(incLayer)

fig


#latList=dfAcc_1['lat'].values.tolist()
#lonList=dfAcc_1['lon'].values.tolist()
#gmap.heatmap( latList, lonList ) 
#gmap.draw( r"C:\Users\70018928\Quantra_Learning\map1.html" ) 


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='1200px'))

In [8]:
dfAcc[['datetime','tweet']].copy()

,date,tweet
0,2020-05-10 08:41:12,08.39 #ซอยรามคำแหง110 แยก25\nเก๋งกับจยย.ชนกัน เสียชีวิต 1 คน การจราจรชะลอตัว #อุบัติเหตุ #รถติด\n\n(Cr.มูลนิธิร่วมกตัญญู) pic.twitter.com/aqgKAIofHS
1,2020-05-10 08:06:44,08.05 #ถนนลาดพร้าว ช่วงแยกแฮปปี้แลนด์ >ซอยลาดพร้าว101 ที่ปากซ.ลาดพร้าว132 รถเมล์จอดเสีย มีคราบน้ำมันหกบนถนนช่องทางซ้าย การจราจรชะลอตัว #รถติด
2,2020-05-10 08:05:21,08.04 #ถนนเพชรเกษม ช่วงแยกหนองแขม >แยกพุทธมณฑลสาย3 เลย ม.เอเชียอาคเนย์ เล็กน้อย เก๋งชนกับจยย. ขวางช่องทางขวา การจราจรเคลื่อนตัวช้า #อุบัติเหตุ #รถติด
3,2020-05-10 07:48:08,07.47 #ถนนสมเด็จพระเจ้าตากสิน ช่วงตากสิน >รพ.ทหารเรือ ที่ปากซอยตากสิน21 รถเมล์จอดเสีย ขวางช่องทางซ้าย การจราจรชะลอตัว #รถติด
4,2020-05-10 07:20:28,รบกวนขอเวลาซัก 1 นาทีขอ 1 นาทีช่วยดูคลิปให้หน่อยค่ะ\n\nขอบคุณค่ะ\n\n https://youtu.be/Lse2_3dxogI \n#รถติด
5,2020-05-10 07:11:20,07:07 #ถนนพุทธสาคร ช่วงต่างระดับคลองมะเดื่อ >เพชรเกษมตัดสาย4 เลยสะพานข้ามคลองภาษีเจริญ 100ม. ปิคอัพชนกับจยย. ขวางช่องทางขวา บาดเจ็บ 1 คน การจราจรชะลอตัว #รถติด #อุบัติเหตุ
